In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install allosaurus

     |████████████████████████████████| 52 kB 834 kB/s 
     |████████████████████████████████| 72 kB 615 kB/s 
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10765 sha256=9baff5f7ad61f7cb9df62e572405ddcf1e1c75f32f9a9bc058f6f99559afab41
  Stored in directory: /root/.cache/pip/wheels/1a/f4/8a/a5024fb77b32ed369e5c409081e5f00fbe3b92fdad653f6e69
Successfully built unicodecsv


In [3]:
#These libraries help to interact with the operating system and the runtime environment respectively
import os
import sys
import pickle

#Model/Training related libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

#Dataloader libraries
from torch.utils.data import DataLoader, Dataset

# Transforms and datasets
import torchvision.transforms as transforms
import torchvision.datasets as dset

import time
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tqdm import tqdm
import random

# Allosaurus
from allosaurus.audio import read_audio
from allosaurus.app import read_recognizer
from allosaurus.am.utils import *

In [4]:
recognizer = read_recognizer()

downloading model  latest
from:  https://github.com/xinjli/allosaurus/releases/download/v1.0/latest.tar.gz
to:    /usr/local/lib/python3.7/dist-packages/allosaurus/pretrained
please wait...


In [5]:
# df = pd.read_csv("/content/gdrive/MyDrive/iemocap_full_dataset.csv")
df = pd.read_csv("/content/gdrive/MyDrive/18786 IDL/IDL Project/iemocap_full_dataset.csv")
df.shape

(10039, 7)

In [6]:
df = df[df.emotion != 'xxx']  # only keep data that has emotion label
# only keep 'neu', 'hap', 'sad', 'ang' labels
df = df.drop(df[~ ((df.emotion == 'neu') | (df.emotion == 'hap') | (df.emotion == 'sad') | (df.emotion == 'ang'))].index)

df_unedit = df.copy()
df_unedit["path"] = df_unedit["path"].apply(lambda x : x.split('/')[-1])
all_files = list(df_unedit.path)
file_to_emotion = dict(zip(df_unedit.path, df_unedit.emotion))

all_full_files = list(df.path)
print(df)
print(df_unedit)
print(len(file_to_emotion))
print(file_to_emotion)
print(all_full_files)

       session  ...                                               path
0            1  ...  Session1/sentences/wav/Ses01F_script02_1/Ses01...
4            1  ...  Session1/sentences/wav/Ses01F_script02_1/Ses01...
6            1  ...  Session1/sentences/wav/Ses01F_script02_1/Ses01...
7            1  ...  Session1/sentences/wav/Ses01F_script02_1/Ses01...
8            1  ...  Session1/sentences/wav/Ses01F_script02_1/Ses01...
...        ...  ...                                                ...
10034        5  ...  Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10035        5  ...  Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10036        5  ...  Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10037        5  ...  Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10038        5  ...  Session5/sentences/wav/Ses05F_impro06/Ses05F_i...

[4490 rows x 7 columns]
       session  method  ... agreement                        path
0            1  script  ...         3  Ses01F_script02_1_

In [7]:
from collections import Counter

# get unique emotions
# emotion_to_label = {'neu': 0, 'fru': 1, 'sad': 2, 'sur': 3, 'ang': 4, 'hap': 5, 'exc': 6, 'fea': 7, 'dis': 8, 'oth': 9}
emotion_to_label = {'neu': 0, 'hap': 1, 'sad': 2, 'ang': 3}
label_to_emotion = {v: k for k, v in emotion_to_label.items()}
print(emotion_to_label)
print(label_to_emotion)

# counter number of class instances
emotion_instances_list = [v for v in file_to_emotion.values()]
counter = Counter(emotion_instances_list)
print(counter)

{'neu': 0, 'hap': 1, 'sad': 2, 'ang': 3}
{0: 'neu', 1: 'hap', 2: 'sad', 3: 'ang'}
Counter({'neu': 1708, 'ang': 1103, 'sad': 1084, 'hap': 595})


In [8]:
file_to_label = {k: emotion_to_label[v] for k, v in file_to_emotion.items()}
print(file_to_label)

{'Ses01F_script02_1_F000.wav': 0, 'Ses01F_script02_1_F004.wav': 0, 'Ses01F_script02_1_F006.wav': 3, 'Ses01F_script02_1_F007.wav': 3, 'Ses01F_script02_1_F008.wav': 0, 'Ses01F_script02_1_F009.wav': 1, 'Ses01F_script02_1_F010.wav': 2, 'Ses01F_script02_1_F015.wav': 0, 'Ses01F_script02_1_F024.wav': 0, 'Ses01F_script02_1_F025.wav': 0, 'Ses01F_script02_1_M007.wav': 0, 'Ses01F_script02_1_M008.wav': 0, 'Ses01F_script02_1_M009.wav': 0, 'Ses01F_script02_1_M010.wav': 0, 'Ses01F_script02_1_M011.wav': 0, 'Ses01F_script02_1_M013.wav': 0, 'Ses01F_script02_1_M014.wav': 0, 'Ses01F_script02_1_M015.wav': 0, 'Ses01F_script02_1_M020.wav': 0, 'Ses01F_script02_1_M021.wav': 0, 'Ses01F_script02_1_M023.wav': 0, 'Ses01F_script02_1_M024.wav': 0, 'Ses01F_script02_1_M025.wav': 0, 'Ses01F_script02_1_M026.wav': 0, 'Ses01F_script02_1_M029.wav': 0, 'Ses01F_script02_1_M031.wav': 0, 'Ses01F_script02_1_M032.wav': 0, 'Ses01F_script02_1_M033.wav': 0, 'Ses01F_script02_1_M034.wav': 0, 'Ses01F_script02_1_M036.wav': 1, 'Ses01F_s

In [9]:
class MyDataset(Dataset):
    def __init__(self, file_list, target_list):
        
        self.file_list = file_list
        self.target_list = target_list
        self.num_classes = len(list(set(target_list)))

        self.x = file_list
        self.y = target_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        filepath = self.file_list[index]
        x = torch.tensor(recognizer.pm.compute(read_audio(filepath)))
        x = x.detach()
        x_len = torch.tensor(np.array([x.shape[0]], dtype=np.int32))
        x_len = x_len.detach()
        y = torch.Tensor([self.target_list[index]])
        return x, x_len, y

In [10]:
%cd /content
!ls

/content
gdrive	sample_data


In [11]:
# collate function
def pad_collate(batch):
    # print("inside collate")
    # batch looks like [(x0, xlen0, y0), (x4, xlen4, y4), (x2, xlen2, y2)... ]
    feats = [sample[0] for sample in batch]
    feat_lens = [sample[1] for sample in batch]
    target_list = torch.Tensor([sample[2] for sample in batch])

    feats = pad_sequence(feats, batch_first=True, padding_value=0) # batch, features, len
    feat_lens = pad_sequence(feat_lens, batch_first=True, padding_value=0).squeeze()
    idx = torch.argsort(feat_lens, descending=True) # sorting the input in descending order as required by the lstms in AM.

    # reorder
    # tensor_batch_feat = feats[idx]
    # tensor_batch_feat_len = feat_lens[idx]
    targets = target_list[idx]
    tensor_batch_feat, tensor_batch_feat_len = move_to_tensor([feats[idx], feat_lens[idx]], device_id=-1) # converting to the required tensors

    # Features
    output_tensor, input_lengths = recognizer.am(tensor_batch_feat, tensor_batch_feat_len, return_lstm=True) # output_shape: [len,batch,features]
    output_tensor = output_tensor.detach()
    input_lengths = input_lengths.detach()
    
    return output_tensor, input_lengths, targets

In [12]:
# all_file_paths = [os.path.join("gdrive", "MyDrive", "data", "IEMOCAP_full_release", file_path) for file_path in all_full_files]
all_file_paths = [os.path.join("gdrive", "MyDrive", "18786 IDL", "IDL Project", "data", "IEMOCAP_full_release", file_path) for file_path in all_full_files]
total_instances = len(all_file_paths)

In [13]:
num_train = round(0.8 * total_instances)
num_test_all = total_instances - num_train
num_val = round(0.5 * num_test_all)
num_test = num_test_all - num_val

print("number training instances:", str(num_train))
print("number validation instances:", str(num_val))
print("number test instances:", str(num_test))
assert(num_train + num_val + num_test == total_instances)

number training instances: 3592
number validation instances: 449
number test instances: 449


In [14]:
# shuffle data
import random
random.seed(2021)

shuffled_data_paths = random.sample(all_file_paths, k=total_instances)
# train_list_paths = shuffled_data_paths[:num_train]
# testall_list_paths = shuffled_data_paths[num_train:]
# val_list_paths = testall_list_paths[:num_val]
# test_list_paths = testall_list_paths[num_test:]

# assert(len(train_list_paths) + len(val_list_paths) + len(test_list_paths) == total_instances)

# train, val, test variables:
# train_list_paths
# val_list_paths
# test_list_paths

##Model

In [15]:
class ICASSP3CNN(nn.Module):
    def __init__(self, embed_size=640, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=31):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        
        batch_size = x.shape[0]
        
        x = x.permute(1, 2, 0)    # (seq_len, batch_size, input_size) -> (batch_size, input_size, seq_len)
      
        cnn_output = torch.cat([self.cnn(x), self.cnn2(x), self.cnn3(x)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]
        
        logits = self.linear(h_n)

        return logits
        
        
class ICASSP2CNN(nn.Module):
    def __init__(self, embed_size=640, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=31):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)

        self.batchnorm = nn.BatchNorm1d(2 * embed_size)

        self.lstm = nn.LSTM(input_size = 2 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        
        batch_size = x.shape[0]
        # torch.Size([468, 64, 640])
        x = x.permute(1, 2, 0)    # (seq_len, batch_size, input_size) -> (batch_size, input_size, seq_len)

        cnn_output = torch.cat([self.cnn(x), self.cnn2(x)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]
        
        logits = self.linear(h_n)

        return logits
    

class ICASSP1CNN(nn.Module):
    def __init__(self, embed_size=640, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=31):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)

        self.batchnorm = nn.BatchNorm1d(embed_size)

        self.lstm = nn.LSTM(input_size = embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        batch_size = x.shape[0]
     
        x = x.permute(1, 2, 0)    # (seq_len, batch_size, input_size) -> (batch_size, input_size, seq_len)

        cnn_output = self.cnn(x)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]
        
        logits = self.linear(h_n)

        return logits
        

## Training

In [16]:
def train_language_model(train_loader_LM, model, opt, criterion, device):

    loss_accum = 0.0
    batch_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader_LM):

        x = x.to(device)
        #lengths = lengths.to(device)
        y = y.long().to(device)
        opt.zero_grad()

        logits = model(x, lengths)
        
        loss = criterion(logits.permute(0,2,1), y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()      

    NLL = loss_accum / batch_cnt
        
    return model, NLL


def train_model(train_loader, model, opt, criterion, device):

    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0     #count correct predictions
    err_cnt = 0     #count incorrect predictions

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):
        x = x.to(device)
        #lengths = lengths.to(device)
        y = y.long().to(device)
        opt.zero_grad()

        logits = model(x, lengths)

        loss = criterion(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
                     
    training_accuracy =  acc_cnt/(err_cnt+acc_cnt) 
    training_loss = loss_accum / batch_cnt
        
    return model, training_accuracy, training_loss


def test_model(loader, model, opt, criterion, device):
    model.eval()
    acc_cnt = 0
    err_cnt = 0

    for x, lengths, y in loader:
        
        x = x.to(device)
        y = y.long().to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    
    return current_acc

## Main runner

In [17]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
model = ICASSP1CNN()
# checkpoint = torch.load("/content/gdrive/MyDrive/model/model.pt")
# model.load_state_dict(checkpoint['model_state_dict'])
# opt.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
criterion = nn.CrossEntropyLoss()
model.to(device)
# opt = optim.Adam(model.parameters(), lr = 0.001)
# criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if cuda else "cpu")
# valid_acc = test_model(val_loader, model, opt, criterion, device)

In [18]:
def get_k_folder(k, i):
  n = num_train+num_val
  train_val_list_paths = shuffled_data_paths[:n]
  fold_size = (n + k-1) // k

  train_list_paths = train_val_list_paths[:i*fold_size] + train_val_list_paths[i*fold_size+fold_size:n]
  val_list_paths = train_val_list_paths[i*fold_size:min(i*fold_size+fold_size, n)]

  return train_list_paths, val_list_paths

In [19]:
def get_loaders(k, i):
  train_list_paths, val_list_paths = get_k_folder(k, i)
  # get corresponding labels for data
  train_list_labels = [file_to_label[filepath.split('/')[-1]] for filepath in train_list_paths]
  val_list_labels = [file_to_label[filepath.split('/')[-1]] for filepath in val_list_paths]

  assert(len(train_list_labels) == len(train_list_paths))
  assert(len(val_list_labels) == len(val_list_paths))

  # train dataloader
  train_dset = MyDataset(train_list_paths, train_list_labels)
  train_args = dict(shuffle=True, batch_size=128, num_workers=2, collate_fn=pad_collate, drop_last=True)  # change to num_workers=4 on diff platform
  train_loader = DataLoader(train_dset, **train_args)

  # val dataloader
  val_dset = MyDataset(val_list_paths, val_list_labels)
  val_args = dict(shuffle=False, batch_size=128, num_workers=2, collate_fn=pad_collate, drop_last=True)
  val_loader = DataLoader(val_dset, **val_args)

  return train_loader, val_loader

In [20]:
def reset_weights(m):
    '''
        Try resetting model weights to avoid
        weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [21]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

run_num = 1
n_epochs = 10
cuda = torch.cuda.is_available()

#Define Training Grid Search
# model_list = ['1CNN', '2CNN', '3CNN']
# emb_size_list = [640, 640, 640]
# hidden_size_list = [128, 256, 384, 512]
# num_lstm_layers_list = [1, 2, 3]
# bidirectional_list = [False, True]

# model_list = ['1CNN', '2CNN', '3CNN']
# emb_size_list = [640]
# hidden_size_list = [128, 256]
# num_lstm_layers_list = [1, 2, 3]
# bidirectional_list = [False]
models = ["3CNN 640 128 2 False", "3CNN 640 256 2 False", "3CNN 640 256 3 False"]


model_id = 0
best_avg_acc = 0

for model in models:
    model_id += 1
    start_time = time.time()

    split_string = model.split(' ')

    model_name = split_string[0]
    emb_size = int(split_string[1])
    hidden_size = int(split_string[2])
    num_lstm = int(split_string[3])
    bidirectionality = True if split_string[4] == "True" else False
    
    #Log Metadata
    metadata = model_name + ',' + str(bidirectionality) + ',' + str(emb_size) + ',' + str(num_lstm) + ',' + str(hidden_size)
    print(metadata)

    #initialize model
    if model_name == '1CNN':
        model = ICASSP1CNN(emb_size, hidden_size, num_lstm, bidirectionality)
    elif model_name == '2CNN':
        model = ICASSP2CNN(emb_size, hidden_size, num_lstm, bidirectionality)
    elif model_name == '3CNN':
        model = ICASSP3CNN(emb_size, hidden_size, num_lstm, bidirectionality)        

    device = torch.device("cuda" if cuda else "cpu")
    model.to(device)
    
    print(model)
    
    max_valid_acc = 0
    max_test_acc = 0
    k = 4

    avg_val_acc = 0
    for i in range(k):
        # start with new opt, criterion and scheduler for each fold
        opt = optim.Adam(model.parameters(), lr = 0.001)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=3, threshold=0.04, threshold_mode='abs', verbose=True)

        # reset weight
        model.apply(reset_weights) 


        print(f'Running {i}th cross validation.......')
        train_loader, val_loader = get_loaders(k, i)
        for n in range(n_epochs):
            model, train_acc, train_loss = train_model(train_loader, model, opt, criterion, device)
            valid_acc = test_model(val_loader, model, opt, criterion, device)
            # test_acc = test_model(test_loader, model, opt, criterion, device)

            if valid_acc > max_valid_acc:
                max_valid_acc = valid_acc
                # max_test_acc = test_acc
            
            scheduler.step(valid_acc)
            if n == n_epochs-1:
                avg_val_acc += valid_acc

            # Print log of accuracy and loss
            print("Fold: "+str(i) + ", Epoch: "+str(n)+", Training Accuracy: "+str(train_acc)+", Training loss:"+str(train_loss)+ ", Validation accuracy:"+str(valid_acc))

    avg_val_acc /= k
    print("The avg acc is: " + str(avg_val_acc))
    # if avg_val_acc >= best_avg_acc:
    save_path = os.path.join("gdrive", "MyDrive", "18786 IDL", "IDL Project", "saved_models", "AESDD", "yifan", f"run{run_num}", f"fold{i}_model{model_name}_embed{emb_size}_hsize{hidden_size}_numlstm{num_lstm}.pth")
    torch.save({'model_id': id,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt.state_dict(),
                'scheduler_state_dict' : scheduler.state_dict(),
                'avg_val_acc': avg_val_acc,
                }, save_path)
    
    total_time = (time.time() - start_time)/60

# for model_name in model_list:
#     for bidirectionality in bidirectional_list:
#         for emb_size in emb_size_list:
#             for num_lstm in num_lstm_layers_list:
#                 for hidden_size in hidden_size_list:
#                         model_id += 1
#                         start_time = time.time()
                        
#                         #Log Metadata
#                         metadata = model_name + ',' + str(bidirectionality) + ',' + str(emb_size) + ',' + str(num_lstm) + ',' + str(hidden_size)
#                         print(metadata)

#                         #initialize model
#                         if model_name == '1CNN':
#                             model = ICASSP1CNN(emb_size, hidden_size, num_lstm, bidirectionality)
#                         elif model_name == '2CNN':
#                             model = ICASSP2CNN(emb_size, hidden_size, num_lstm, bidirectionality)
#                         elif model_name == '3CNN':
#                             model = ICASSP3CNN(emb_size, hidden_size, num_lstm, bidirectionality)        
 
#                         device = torch.device("cuda" if cuda else "cpu")
#                         model.to(device)
                        
#                         print(model)
                        
#                         max_valid_acc = 0
#                         max_test_acc = 0
#                         k = 4

#                         avg_val_acc = 0
#                         for i in range(k):
#                           # start with new opt, criterion and scheduler for each fold
#                           opt = optim.Adam(model.parameters(), lr = 0.001)
#                           criterion = nn.CrossEntropyLoss()
#                           scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=3, threshold=0.04, threshold_mode='abs', verbose=True)

#                           # reset weight
#                           model.apply(reset_weights) 


#                           print(f'Running {i}th cross validation.......')
#                           train_loader, val_loader = get_loaders(k, i)
#                           for n in range(n_epochs):
#                               model, train_acc, train_loss = train_model(train_loader, model, opt, criterion, device)
#                               valid_acc = test_model(val_loader, model, opt, criterion, device)
#                               # test_acc = test_model(test_loader, model, opt, criterion, device)

#                               if valid_acc > max_valid_acc:
#                                   max_valid_acc = valid_acc
#                                   # max_test_acc = test_acc
                              
#                               scheduler.step(valid_acc)
#                               if n == n_epochs-1:
#                                 avg_val_acc += valid_acc

#                               # Print log of accuracy and loss
#                               print("Fold: "+str(i) + ", Epoch: "+str(n)+", Training Accuracy: "+str(train_acc)+", Training loss:"+str(train_loss)+ ", Validation accuracy:"+str(valid_acc))

#                         avg_val_acc /= k
#                         print("The avg acc is: " + str(avg_val_acc))
#                         if avg_val_acc >= best_avg_acc:
#                           save_path = os.path.join("gdrive", "MyDrive", "18786 IDL", "IDL Project", "saved_models", "AESDD", "yifan", f"run{run_num}", f"fold{i}_model{model_name}_embed{emb_size}_hsize{hidden_size}_numlstm{num_lstm}.pth")
#                           torch.save({
#                                       'model_id': id,
#                                       'model_state_dict': model.state_dict(),
#                                       'optimizer_state_dict': opt.state_dict(),
#                                       'scheduler_state_dict' : scheduler.state_dict(),
#                                       }, save_path)
                        
#                         total_time = (time.time() - start_time)/60
            

3CNN,False,640,2,128
ICASSP3CNN(
  (cnn): Conv1d(640, 640, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(640, 640, kernel_size=(5,), stride=(1,), padding=(2,))
  (cnn3): Conv1d(640, 640, kernel_size=(7,), stride=(1,), padding=(3,))
  (batchnorm): BatchNorm1d(1920, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(1920, 128, num_layers=2)
  (linear): Linear(in_features=128, out_features=31, bias=True)
)
Reset trainable parameters of layer = Conv1d(640, 640, kernel_size=(3,), stride=(1,), padding=(1,))
Reset trainable parameters of layer = Conv1d(640, 640, kernel_size=(5,), stride=(1,), padding=(2,))
Reset trainable parameters of layer = Conv1d(640, 640, kernel_size=(7,), stride=(1,), padding=(3,))
Reset trainable parameters of layer = BatchNorm1d(1920, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = LSTM(1920, 128, num_layers=2)
Reset trainable parameters of layer = Linear(in_features=128,

In [ ]:
def get_test_acc(model, test_loader):
    # checkpoint = torch.load(model_pth)
    # model.load_state_dict(checkpoint["model_state_dict"])

    model.eval()
    test_num_correct = 0
    total = 0
    for batch_num, (x, lengths, y) in enumerate(test_loader):
        x = x.to(device)
        y = y.long().to(device)

        logits = model(x, lengths)
        test_num_correct += (torch.argmax(logits, axis=1) == y).sum().item()
        total += len(y)

    test_acc = test_num_correct / total
    return test_acc

In [ ]:
# test dataloader
test_list_labels = [file_to_label[filepath.split('/')[-1]] for filepath in test_list_paths]
test_dset = MyDataset(test_list_paths, test_list_labels)
test_args = dict(shuffle=False, batch_size=32, num_workers=2, collate_fn=pad_collate, drop_last=True)
test_loader = DataLoader(test_dset, **test_args)

test_acc = get_test_acc(model, test_loader)
print(test_acc)

0.5625
